In [1]:
!pip install transformers

In [2]:
!pip install SentencePiece

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import json

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-multitask")
model = AutoModelForSeq2SeqLM.from_pretrained('question_generation_model_1')

C:\Users\totmi\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
device

device(type='cuda')

In [4]:
with open('TrainOnlySentenceJson.json', 'r', encoding='utf-8-sig',) as f:
    test_data = json.load(f)

In [5]:
!pip install tqdm

In [6]:
from tqdm import tqdm
import gc
predictions = []
batch_size = 32  # задаем размер пакета


with torch.no_grad():
    for i in tqdm(range(0, len(test_data), 8)):
        batch = test_data[i:i+8]

        input_sentences = [item['text'] for item in batch]
        input_tokens = tokenizer.batch_encode_plus(input_sentences, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
        input_ids = input_tokens['input_ids'].to(device)
        attention_mask = input_tokens['attention_mask'].to(device)

        # Генерация предсказания
        output_tokens = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=512,
            num_beams=4,
            early_stopping=True
        )

        # Декодирование предсказания и добавление в список
        for j in range(input_ids.shape[0]):
            predicted_question = tokenizer.decode(output_tokens[j], skip_special_tokens=True)
            predictions.append({'text': input_sentences[j], 'predicted_question': predicted_question})

        # Очистка памяти графического процессора
        torch.cuda.empty_cache()

100%|██████████| 31250/31250 [8:20:11<00:00,  1.04it/s]   


In [7]:
with open('predictions.json', 'w', encoding='utf-8') as f:
  json.dump(predictions, f, ensure_ascii=False, indent=4)